# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iqaluit,63.7506,-68.5145,10.85,87,100,2.06,CA,1723164310
1,1,nemuro,43.3236,145.5750,19.86,95,100,4.22,JP,1723164311
2,2,kafin hausa,12.2375,9.9080,24.95,80,100,4.58,NG,1723164312
3,3,kingston,17.9970,-76.7936,30.53,66,20,7.20,JM,1723164313
4,4,utrik,11.2278,169.8474,28.22,80,83,7.92,MH,1723164315


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat',
    size='Humidity',  # Size of the points based on humidity
    title='City Humidity Map',
    geo=True,         # Enable geographic plotting
    tiles='OSM'       # Use OpenStreetMap tiles for the base map
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 29)
                                       & (city_data_df["Max Temp"] > 22) 
                                       & (city_data_df["Cloudiness"] < 2)
                                       & (city_data_df["Wind Speed"] < 2)
                                      ]

# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
60,60,vila franca do campo,37.7167,-25.4333,22.23,99,0,1.91,PT,1723164382
165,165,labuan bajo,-8.4964,119.8877,27.13,67,0,0.73,ID,1723164507
174,174,kas,36.2018,29.6377,27.40,65,1,1.08,TR,1723164519
225,225,verkhnevilyuysk,63.4453,120.3167,22.28,45,0,1.31,RU,1723164579
253,253,praia da vitoria,38.7333,-27.0667,23.14,88,0,1.15,PT,1723164613
309,309,abdurahmoni jomi,37.9464,68.8088,22.16,64,0,1.00,TJ,1723164685
352,352,archangelos,36.2149,28.1149,25.99,74,0,1.74,GR,1723164739
368,368,gaza,31.5000,34.4667,25.74,75,0,0.81,PS,1723164759
396,396,beauzelle,43.6671,1.3752,23.06,60,0,1.54,FR,1723164793
403,403,farsala,39.2947,22.3847,23.58,58,0,0.87,GR,1723164801


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
60,vila franca do campo,PT,37.7167,-25.4333,99,
165,labuan bajo,ID,-8.4964,119.8877,67,
174,kas,TR,36.2018,29.6377,65,
225,verkhnevilyuysk,RU,63.4453,120.3167,45,
253,praia da vitoria,PT,38.7333,-27.0667,88,
309,abdurahmoni jomi,TJ,37.9464,68.8088,64,
352,archangelos,GR,36.2149,28.1149,74,
368,gaza,PS,31.5000,34.4667,75,
396,beauzelle,FR,43.6671,1.3752,60,
403,farsala,GR,39.2947,22.3847,58,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000 
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vila franca do campo - nearest hotel: No hotel found
labuan bajo - nearest hotel: No hotel found
kas - nearest hotel: No hotel found
verkhnevilyuysk - nearest hotel: No hotel found
praia da vitoria - nearest hotel: No hotel found
abdurahmoni jomi - nearest hotel: No hotel found
archangelos - nearest hotel: No hotel found
gaza - nearest hotel: No hotel found
beauzelle - nearest hotel: No hotel found
farsala - nearest hotel: No hotel found
zakynthos - nearest hotel: No hotel found
boumahra ahmed - nearest hotel: No hotel found
sudak - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
60,vila franca do campo,PT,37.7167,-25.4333,99,No hotel found
165,labuan bajo,ID,-8.4964,119.8877,67,No hotel found
174,kas,TR,36.2018,29.6377,65,No hotel found
225,verkhnevilyuysk,RU,63.4453,120.3167,45,No hotel found
253,praia da vitoria,PT,38.7333,-27.0667,88,No hotel found
309,abdurahmoni jomi,TJ,37.9464,68.8088,64,No hotel found
352,archangelos,GR,36.2149,28.1149,74,No hotel found
368,gaza,PS,31.5000,34.4667,75,No hotel found
396,beauzelle,FR,43.6671,1.3752,60,No hotel found
403,farsala,GR,39.2947,22.3847,58,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )
# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)